In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re, textwrap
import pandas as pd
import numpy as np
import argparse

In [2]:
!pip install TwitterAPI

In [3]:
#NOTE: Must have TwitterAPI Installed
from TwitterAPI import TwitterAPI
from TwitterAPI import TwitterPager

In [4]:
api_key="LhCSedmn8rIMMipticrS4g350"
api_secret_key="y4Ws7XPuV9cmsOS3aId4LPIS5kMiA7nFzldBblOZo5lxfmC032"
bearer="AAAAAAAAAAAAAAAAAAAAAB91GAEAAAAA24kbznYamgCYk%2FRMiXyttUXILIU%3DIazdeos2q8xEOP4GkXAaK6gX37u7lx0Gq6hrg9MT1z6mtAMDgO"
AccessToken="1285002674597851137-IXKri9EFCqh9EozdKZqTqKL7BSXDWI"
AccessTokenSecret="BjJM0o0X1hZ8hjUwYDUfeSzWrSPJjgr1D5wXfGWQeZh8j"

In [5]:
# Setup Twitter API Client
api = TwitterAPI(api_key, 
                 api_secret_key,
                 AccessToken,
                 AccessTokenSecret)

In [6]:
def remove_non_ascii_2(text):
    """
    helper function to remove non-ascii character like emojis and things
    """
    return re.sub(r'[^\x00-\x7F]+', "'", text)

def extract_tweet_info(item, count, term):
    """
    Extract columns for each tweet
    """
    # Variables
    time = item['created_at'] # might need to use python to format
    tweet = remove_non_ascii_2(item['text']) #tweet text
    search_term = term # search term
    language = item['lang'] # language of the tweet
    retweet_count = item['retweet_count'] # retweet count
    user_id = item['user']['id'] # user id of the author
    is_verified_user = item['user']['verified'] # is the user verified (binary)
    user_friend_count = item['user']['friends_count'] # number of author's friends lol
    user_follower_count = item['user']['followers_count'] # number of author's followers
    user_location = item['user']['location']
    
    # clean up hashtags to only extract text
    hashtags = item['entities']['hashtags']
    hashtags_result = []
    for x in hashtags:
        hashtags_result.append(x['text'])

    #Add Row to Data Frame
    new_row = {"created_at":time, 
               "tweet":tweet, 
               "search_term": search_term, 
               "hashtags":str(hashtags_result).strip('[]'), 
               "language":language,
               "retweet_count":retweet_count,
               "user_id":user_id,
               "is_verified_user":is_verified_user,
               "user_follower_count":user_friend_count,
               "user_friend_count":user_follower_count,
               "user_location":user_location
              }
    
    return new_row

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

Returns tweets by users located within a given radius of the given latitude/longitude. The location is preferentially taking from the Geotagging API, but will fall back to their Twitter profile.

In [7]:
# TODO: Update these with the right parameters
limit = 200
SEARCH_TERM = 'facemask','COVID19'
location_box = '29,-95'

# create a Twitter paging client, will request one page at a time
r = TwitterPager(api, 
                 'search/tweets', 
                 {'q': SEARCH_TERM, 
                  'locations':location_box})

In [8]:
#Setup Initial Empty Data Frame
header = [
          "created_at", 
          "tweet", 
          "search_term", 
          "hashtags", 
          "language",
          "retweet_count",
          "user_id",
          "is_verified_user",
          "user_follower_count",
          "user_friend_count",
          "user_location",
         ]
index = np.arange(0)
df = pd.DataFrame(columns=header, index = index)

In [9]:
count = 0 # counter
print("requested tweets for hashtag is limited to {} tweets".format(limit))
for item in r.get_iterator(wait=6):
    if 'text' in item:
        if count <= limit:
            if count % 10 == 0:
                print("collecting tweet {} of {}...".format(count, limit))
            count += 1
            #Extract Tweet Info
            new_row = extract_tweet_info(item, count, SEARCH_TERM)
            df = df.append(new_row, ignore_index=True)

        else:
            print("requested tweet limit reached...")
            print("ending query for hashtag...")
            break

    elif 'message' in item and item['code'] == 88:
        print('SUSPEND, RATE LIMIT EXCEEDED: %s' % item['message'])
        break

requested tweets for hashtag is limited to 2 tweets
collecting tweet 0 of 2...
requested tweet limit reached...
ending query for hashtag...


In [10]:
df.head()

,created_at,tweet,search_term,hashtags,language,retweet_count,user_id,is_verified_user,user_follower_count,user_friend_count,user_location
0,Wed Sep 23 22:12:07 +0000 2020,RT @natalia_linn: Japanese Plum Blossom on Wav...,"(facemask, COVID19)","'facemask', 'cute', 'floralfacemask'",en,1,194275025,False,1048,1734,San Diego
1,Wed Sep 23 22:11:48 +0000 2020,"RT @SantaFeKiss: Embroidered, Washable, 3 laye...","(facemask, COVID19)",,en,6,890239500814872577,False,2223,4098,
2,Wed Sep 23 22:10:10 +0000 2020,@AMartelaer @vrtpano @groen Vertaling voor Eng...,"(facemask, COVID19)",,en,0,35852000,False,4411,590,"Shannon, Co. Clare, Ireland"


In [20]:
# save the dataframe to csv
print(df.shape)
df.to_csv("twitter_dataTX817.csv")

(251, 11)
